# Arrow detection using tensorflow

## 1. Create folder hierarchy

In [12]:
import os

In [99]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
# CUSTOM_MODEL_NAME = 'my_centernet_mobilenetv2fpn' 
# PRETRAINED_MODEL_NAME = 'centernet_mobilenetv2_fpn_kpts'
# PRETRAINED_MODEL_URL = 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1?tf-hub-format=compressed'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),

    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': TF_RECORD_SCRIPT_NAME, 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# create workspace directories
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}

## 2. Download pretrained model

In [100]:
#!wget {PRETRAINED_MODEL_URL} -O {PRETRAINED_MODEL_NAME}.tar.gz
!mv {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

mv: cannot stat 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz': No such file or directory
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


## 3. Clone tensorflow and build models API

In [11]:
!git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

fatal: destination path 'Tensorflow/models' already exists and is not an empty directory.


In [ ]:
# Build API buffers
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection
!cp object_detection/packages/tf2/setup.py . && python3 -m pip install . 

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python3 {VERIFICATION_SCRIPT}

In [44]:
import object_detection

## 4. Create Label Map

In [30]:
labels = [{'name':'10', 'id':1},
          {'name':'9', 'id':2},
          {'name':'8', 'id':3},
          {'name':'7', 'id':4},
          {'name':'6', 'id':5},
          {'name':'5', 'id':6},
          {'name':'4', 'id':7},
          {'name':'3', 'id':8},
          {'name':'2', 'id':9},
          {'name':'1', 'id':10},
          {'name':'M', 'id':11},
          {'name':'X', 'id':12}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [21]:
import glob

DATA='/home/vajicek/data/targets/all/'

annotation_jsons = glob.glob(DATA + '*.json')
test_size = int(len(annotation_jsons) / 3)
test = annotation_jsons[:test_size]
train = annotation_jsons[test_size:]

for test1 in test:
    !cp {test1} {os.path.join(paths['IMAGE_PATH'], 'test')}
    !cp {os.path.splitext(test1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'test')}
for train1 in train:
    !cp {train1} {os.path.join(paths['IMAGE_PATH'], 'train')}
    !cp {os.path.splitext(train1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'train')}

## 5. Create TF records

In [26]:
!python3 {files['TF_RECORD_SCRIPT']} \
    -j {os.path.join(paths['IMAGE_PATH'], 'train')} \
    -l {files['LABELMAP']} \
    -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python3 {files['TF_RECORD_SCRIPT']} \
    -j {os.path.join(paths['IMAGE_PATH'], 'test')} \
    -l {files['LABELMAP']} \
    -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

2021-08-29 17:40:44.651717: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
2021-08-29 17:40:47.413330: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## 6. Copy Model Config to Training Folder

In [101]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} \
    {os.path.join(paths['CHECKPOINT_PATH'])}

## 7. Update Config For Transfer Learning

In [102]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [103]:
# config from pretrained model
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [104]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    text_format.Merge(f.read(), pipeline_config)

In [90]:
#pipeline_config.model.center_net.keypoint_estimation_task.pop()
#pipeline_config.model.center_net.keypoint_estimation_task)

IndexError: list index (-1) out of range

In [110]:
# update model training parameters
# pipeline_config.model.center_net.num_classes = len(labels)
# pipeline_config.model.center_net.keypoint_label_map_path = files['LABELMAP']
#pipeline_config.model.center_net.keypoint_estimation_task=[]

pipeline_config.model.ssd.num_classes = len(labels)

pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'],
                                                                 PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'],
                                                                                        'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'],
                                                                                          'test.record')]

In [111]:
#store updated config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

## 8. Train the Model

In [107]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [108]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'])

In [113]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000


In [112]:
!{command}

2021-08-29 18:40:00.409455: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-29 18:40:02.336119: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-29 18:40:02.338576: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 18:40:02.339033: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 560 computeCapability: 2.1
coreClock: 1.66GHz coreCount: 7 deviceMemorySize: 959.50MiB deviceMemoryBandwidth: 119.45GiB/s
2021-08-29 18:40:02.339057: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-29 18:40:02.342338: I tensorflow/stream_executor/platform/

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0829 18:40:08.351294 140351247476544 deprecation.py:330] From /home/vajicek/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0829 18:40:11.005074 140351247476544 deprecation.py:330] From /home/vajicek/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:206: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0829 18:4